In [18]:
import pandas as pd
import numpy as np
from nltk.translate.bleu_score import corpus_bleu
from sklearn.utils import shuffle
import nltk

### Building the Vocabulary

First, we build the vocabulary dictionaries for the source and target. 
The vocabulary is the the file `vocab.txt` (generated in the other script).

In [6]:
# Word string -> ID mapping
dictionary = dict()

vocabulary_size = len(dictionary)
with open('data/vocab.txt', encoding='utf-8') as f:
    for line in f:
        # disregard the new line aka `\n`
        dictionary[line[:-1]] = len(dictionary)
        
vocabulary_size = len(dictionary)
reverse_dictionary = dict(zip(dictionary.values(),dictionary.keys()))

print('Dictionary:', list(dictionary.items())[:10], end = '\n')
print('Reverse dictionary:', list(reverse_dictionary.items())[:10], end = '\n')
print('Vocabulary size: ', vocabulary_size, end = '\n')


Dictionary: [('<unk>', 0), ('<s>', 1), ('</s>', 2), ('.', 3), ('the', 4), (',', 5), ('a', 6), ('to', 7), ('and', 8), ('i', 9)]
Reverse dictionary: [(0, '<unk>'), (1, '<s>'), (2, '</s>'), (3, '.'), (4, 'the'), (5, ','), (6, 'a'), (7, 'to'), (8, 'and'), (9, 'i')]
Vocabulary size:  50000


### Loading the Dataset
Here we load the data from the `dataset.csv` file (generated in the other script)

In [3]:
dataset = pd.read_csv('data/dataset.csv')

### Data pre-processing
Transform to lower, remove the new line and the punctuation

In [9]:
def lowerDataset(data):
    return data.str.lower() 
    
def cleanDataset(data):
    return data.str.replace('/r/','')                  \
                .str.replace(')','', regex=False)      \
                .str.replace('(','', regex=False)      \
                .str.replace(']','', regex=False)      \
                .str.replace('[','', regex=False)      \
                .str.replace('!','')                   \
                .str.replace('"','')                   \
    
def paddDataset(data):
    return data.str.replace(',', ' ,')                 \
                .str.replace('.',' . ', regex=False)    \
                .str.replace('?',' ?', regex=False)    \
                .str.replace('\n',' ')

In [10]:
wt = nltk.tokenize.WhitespaceTokenizer()
for column in dataset.columns:    
    dataset[column] = lowerDataset(dataset[column]) 
    dataset[column] = cleanDataset(dataset[column])
    dataset[column] = paddDataset(dataset[column])                                    
    dataset[column] = dataset[column].apply(wt.tokenize)
dataset = shuffle(dataset)

In [11]:
dataset.head()

,question,answer
1050529,"[reddit, ,, show, me, one, of, your, favorite,...","[i, was, trying, to, draw, dsotm, during, an, ..."
958524,"[for, those, who, grew, up, poor, ,, what, did...","[until, the, age, of, 12, ,, i, thought, that,..."
405948,"[where, the, hell, is, a, 30, year, old, suppo...","[you're, in, law, school, ., can, you, try, to..."
216595,"[what, job, is, commonly, forgot, about, but, ...","[i, test, food, at, the, factory, to, make, su..."
773802,"[you, are, stranded, on, an, island, with, not...","[step, one, is, to, examine, my, surroundings,..."


### Data analysis
Mean sentence length and standard deviation of sentence length

In [12]:
print('(Questions) Average sentence length: ', dataset['question'].str.len().mean())
print('(Questions) Standard deviation of sentence length: ', dataset['question'].str.len().std())

print('(Answers) Average sentence length: ', dataset['answer'].str.len().mean())
print('(Answers) Standard deviation of sentence length: ', dataset['answer'].str.len().std())

(Questions) Average sentence length:  17.116929628176226
(Questions) Standard deviation of sentence length:  9.138660652379489
(Answers) Average sentence length:  54.4734629738071
(Answers) Standard deviation of sentence length:  844.5689123041486


### Update the sentences to fixed length
Update all sentences with a fixed size, to process the sentences as batches.

In [14]:
max_sent_length = {'question' : 30, 'answer': 70}

def padding_sent(source):
    padded = []
    for tokens in dataset[source]: 
        # adding the start token
        tokens.insert(0, '<s>')  

        if len(tokens) >= max_sent_length[source]:
            tokens = tokens[:(max_sent_length[source] - 1)]
            tokens.append('</s>')

        if len(tokens) < max_sent_length[source]:
            tokens.extend(['</s>' for _ in range(max_sent_length[source] - len(tokens))])  

        padded.append(tokens)
    return padded

In [15]:
questions = padding_sent('question')
answers = padding_sent('answer')

### Create the reverse dataset

In [19]:
def create_reverse_dataset(source):
    reverse_tokens = []
    reverse_dataset = []
    for tokens in source: 
        for token in tokens: 
            if token not in dictionary.keys():
                reverse_tokens.append(dictionary['<unk>'])
            else:
                reverse_tokens.append(dictionary[token])
        reverse_dataset.append(reverse_tokens)
        reverse_tokens = []
    return reverse_dataset

train_inputs =  np.array(create_reverse_dataset(questions), dtype=np.int32)
train_outputs =  np.array(create_reverse_dataset(answers), dtype=np.int32)